In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import sklearn
import swifter
import numpy as np
import lightgbm
from sklearn.model_selection import train_test_split
import joblib
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.linear_model import LogisticRegression, Ridge, RidgeClassifier

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [2]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

In [3]:
rheed_long = pd.read_csv("../data/onti_p.csv")

In [4]:
#проверка на корректную склейку
print(rheed_long.shape)
rheed_long[:1]

(2363772, 18)


,Unnamed: 0,Unnamed: 0.1,Coolness_RHEED,Length_RHEED,R FWHM_RHEED,Speed_RHEED,Time_RHEED,X FWHM_RHEED,Y FWHM_RHEED,Crystal Position,Displayed Rate,Filtered Rate,Fundamental Freq,Length,Raw Rate,Source Power,Speed,Time
0,0,0.0,0.0,0.003721,0.0,0.040601,2021-02-23 02:21:05,0.0,0.0,1.0,6.025618,5.999182,5.967367e+06,0.003721,5.987163,21.635099,NaN,2021-02-23 02:21:05


In [5]:
#rheed_long = df_comma_to_float(rheed_long)
rheed_long.shape

(2363772, 18)

In [6]:
#важные колонки для использования
val_col = ['Coolness_RHEED',  'R FWHM_RHEED',  'X FWHM_RHEED', 'Y FWHM_RHEED',
        'Filtered Rate', 'Displayed Rate', 'Raw Rate',
       'Source Power', 'Crystal Position']


In [7]:
#создаем датасет разными аггрегационными функциями
def generated_dataset(series, wind_size=5):
    result = series.rolling(wind_size).agg(["mean", "std", "var"])
    print(result.shape) 
    delta = series - series.shift(wind_size)
    print(delta.shape)
    result = result.join(delta)
    result.rename(columns={c:str(c)+"_"+str(wind_size) for c in result.columns}, inplace=True)

    print(result.shape)

    return result

In [8]:
df_X = generated_dataset(rheed_long[val_col])
df_X.shape, df_X.columns

(2363772, 27)
(2363772, 9)


/home/ec2-user/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


(2363772, 36)


((2363772, 36),
 Index(['('Coolness_RHEED', 'mean')_5', '('Coolness_RHEED', 'std')_5',
        '('Coolness_RHEED', 'var')_5', '('R FWHM_RHEED', 'mean')_5',
        '('R FWHM_RHEED', 'std')_5', '('R FWHM_RHEED', 'var')_5',
        '('X FWHM_RHEED', 'mean')_5', '('X FWHM_RHEED', 'std')_5',
        '('X FWHM_RHEED', 'var')_5', '('Y FWHM_RHEED', 'mean')_5',
        '('Y FWHM_RHEED', 'std')_5', '('Y FWHM_RHEED', 'var')_5',
        '('Filtered Rate', 'mean')_5', '('Filtered Rate', 'std')_5',
        '('Filtered Rate', 'var')_5', '('Displayed Rate', 'mean')_5',
        '('Displayed Rate', 'std')_5', '('Displayed Rate', 'var')_5',
        '('Raw Rate', 'mean')_5', '('Raw Rate', 'std')_5',
        '('Raw Rate', 'var')_5', '('Source Power', 'mean')_5',
        '('Source Power', 'std')_5', '('Source Power', 'var')_5',
        '('Crystal Position', 'mean')_5', '('Crystal Position', 'std')_5',
        '('Crystal Position', 'var')_5', 'Coolness_RHEED_5', 'R FWHM_RHEED_5',
        'X FWHM_RHEED_5', '

In [9]:
df_X = df_X.join(generated_dataset(rheed_long[val_col],wind_size=50))#,rsuffix="_100_")
df_X.shape

(2363772, 27)
(2363772, 9)
(2363772, 36)


(2363772, 72)

In [10]:
df_X = df_X.join(generated_dataset(rheed_long[val_col],wind_size=500))#,rsuffix="_500_")
df_X.shape

(2363772, 27)
(2363772, 9)
(2363772, 36)


(2363772, 108)

In [11]:
df_X = df_X.join(generated_dataset(rheed_long[val_col],wind_size=1500))#,rsuffix="_1500_")
df_X.shape

(2363772, 27)
(2363772, 9)
(2363772, 36)


(2363772, 144)

In [12]:
mask_base = df_X.isna().sum(axis=1)==0
#df_X.fill(inplace=True)
df_X.shape

(2363772, 144)

In [13]:
mean_coolness = rheed_long.Coolness_RHEED.rolling(50).mean()
y1 = ((mean_coolness - mean_coolness.shift(-50))>5)
y1.value_counts()

False    2343613
True       20159
Name: Coolness_RHEED, dtype: int64

In [14]:
df_X['Anomaly'] = rheed_long['Coolness_RHEED']
df_X['Anomaly'] = (df_X['Anomaly'] - df_X['Anomaly'].rolling(50).mean()).apply(lambda x: (1 if abs(x) >= 5 else 0))

temp = df_X[11000:len(df_X.index)-1000]
anom_indexes = temp[temp['Anomaly'] == 1].index
norm_indexes = temp[temp['Anomaly'] == 0].index

In [15]:
wind_size = 435

df_X.drop('Anomaly', axis=1, inplace=True)
df = df_X[wind_size:-wind_size]

cool_col = '(\'Coolness_RHEED\', \'mean\')_50'
y = ((df[cool_col] - df[cool_col].shift(-wind_size))>5)

# mask = mask_base #&(y>30)
# df = df[mask]
# y = y[mask]
categorical_features_indices = np.where(df.dtypes != np.float)[0]

In [16]:
from sklearn.preprocessing import MinMaxScaler
num_data = len(rheed_long.index)

train_split = 1
num_train = int(train_split * num_data)
num_test = num_data - num_train

train = df_X

num_x_signals = train.shape[1]

def get_data_entry(length=1000):
    # Allocate a new array for the batch of input-signals.
    x_shape = (length, 10000, num_x_signals)
    x_batch = np.zeros(shape=x_shape, dtype=np.float32)

    # Allocate a new array for the batch of output-signals.
    y_shape = (length, 1, 1)
    y_batch = np.zeros(shape=y_shape, dtype=np.float32)

    # Fill the batch with random sequences of data.
    for i in range(length):
        # Get a random start-index.
        # This points somewhere into the training-data.
        idx = random.choice(anom_indexes) if i % 2 == 0 else random.choice(norm_indexes)
            
        # Copy the sequences of data starting at this index.
        x_batch[i] = train[idx - 10000:idx]
        # y_mean = train[idx + 385:idx + 435][0].mean()
#         y_batch[i] = [1 if (abs(train[idx + 435][0] - y_mean) >= 5) else 0]
        y_batch[i] = [1 if i % 2 == 0 else 0]
        
    return (x_batch, y_batch)


In [21]:
import random
x_batch, y_batch = get_data_entry(length=500)
X_train, X_test, y_train, y_test = train_test_split(x_batch, y_batch, test_size=0.25, shuffle=True)

ValueError: Found input variables with inconsistent numbers of samples: [144, 500]

In [18]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=10,
        eval_metric='AUC',
        random_seed=42,
        verbose=False,
        loss_function='Logloss',
    )
    
    cv_data = cv(
        Pool(X_train, y_train, cat_features=categorical_features_indices),
        model.get_params()
    )
    best_accuracy = np.max(cv_data['test-AUC-mean'])
    
    return 1 - best_accuracy # as hyperopt minimises

Failed to load cloudpickle, try installing cloudpickle via "pip install cloudpickle" for enhanced pickling support.


In [19]:
from numpy.random import RandomState

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
#     'colsample_bylevel': hyperopt.hp.uniform('colsample_bylevel', 0.5, 1.0),
#     'bagging_temperature': hyperopt.hp.uniform('bagging_temperature', 0.0, 100),
#     'random_strength': hyperopt.hp.uniform('random_strength', 0.0, 100),
#     'n_estimators': hyperopt.hp.choice('n_estimators', np.arange(50, 250, 25)),
#     'max_depth': hyperopt.hp.choice('max_depth', np.arange(5, 11)),
#     'reg_lambda': hyperopt.hp.uniform('reg_lambda', 1, 10),
#     'scale_pos_weight': hyperopt.hp.uniform('scale_pos_weight', 1, 50)
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=150,
    trials=trials,
    rstate=RandomState(42)
)

print(best)

  0%|          | 0/150 [00:00<?, ?trial/s, best loss=?]

job exception: Input data has invalid shape: (375, 10000, 144). Must be 2 dimensional



  0%|          | 0/150 [00:00<?, ?trial/s, best loss=?]


CatBoostError: Input data has invalid shape: (375, 10000, 144). Must be 2 dimensional

In [20]:
model = CatBoostClassifier(
    #l2_leaf_reg=int(best['l2_leaf_reg']),
    #learning_rate=best['learning_rate'],
    iterations=15000,
    eval_metric='AUC',
    random_seed=42,
    verbose=True,
    loss_function='Logloss',
    od_type='Iter',
    od_wait=150
)

train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

model.fit(train_pool, eval_set=validate_pool)

CatBoostError: Input data has invalid shape: (375, 10000, 144). Must be 2 dimensional

In [ ]:
fpr, tpr, _ = sklearn.metrics.roc_curve(y_test, model.predict_proba(X_test[model_cols])[:,1])
roc_auc = sklearn.metrics.roc_auc_score(y_test, model.predict_proba(X_test[model_cols])[:,1])
plt.figure()
lw = 2
plt.plot(fpr, tpr,color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
[1.0 if i == 'True' else 0.0 for i in model.predict(X_test[model_cols])]

In [ ]:
roc_auc

In [ ]:
import pickle
with open("../saves/catboost6.pkl","wb") as f:
    pickle.dump(model, f)